In [68]:
#loading packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from scipy.stats import t, shapiro, kstest
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [87]:
#loading data set and transforming
data1 = pd.read_csv('data/simko2_data/passport_prots.csv', index_col=0)  
data1.index = data1.index.astype(str)
data1


,22RV1,23132-87,42-MG-BA,451Lu,5637,639-V,647-V,697,769-P,786-0,...,WSU-NHL,YAPC,YH-13,YKG-1,YMB-1-E,YT,ZR-75-30,huH-1,no-10,no-11
protein,,,,,,,,,,,,,,,,,,,,,
A1CF,2.09,2.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.12,NaN,NaN
A2M,2.38,NaN,2.58,NaN,2.79,2.07,3.96,NaN,2.78,2.95,...,0.61,3.00,2.21,2.41,2.45,3.12,2.62,1.06,3.75,3.38
A2ML1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.20,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAAS,3.05,3.18,2.69,4.21,4.07,3.21,3.76,3.15,3.44,2.55,...,3.46,2.87,3.72,3.36,NaN,2.97,3.22,3.75,3.74,3.02
AACS,2.37,1.42,2.50,3.49,NaN,1.96,NaN,NaN,2.08,NaN,...,NaN,NaN,1.70,NaN,3.44,3.70,2.00,-1.34,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWILCH,2.38,2.08,1.61,NaN,NaN,2.88,NaN,2.68,2.77,1.70,...,1.84,1.02,1.63,3.00,NaN,2.08,NaN,NaN,NaN,3.48
ZWINT,1.69,1.85,1.74,NaN,NaN,2.11,NaN,2.27,2.20,1.82,...,2.00,NaN,1.97,2.98,NaN,1.96,1.18,NaN,1.63,2.71
ZYX,3.60,4.74,5.95,4.34,1.40,6.03,2.50,3.24,7.01,6.61,...,4.78,2.44,7.71,5.62,3.90,6.10,4.56,3.59,6.23,6.07


In [70]:
#removing all the proteins with 80% nan values
row_nan_count = data1.isna().sum(axis=1)
row_nan_percent = (row_nan_count/data1.shape[1])*100

#selecting rows with under 80% nan specifically
data1_filt = data1[row_nan_percent < 75]
data1_filt
    

,22RV1,23132-87,42-MG-BA,451Lu,5637,639-V,647-V,697,769-P,786-0,...,WSU-NHL,YAPC,YH-13,YKG-1,YMB-1-E,YT,ZR-75-30,huH-1,no-10,no-11
protein,,,,,,,,,,,,,,,,,,,,,
A2M,2.38,NaN,2.58,NaN,2.79,2.07,3.96,NaN,2.78,2.95,...,0.61,3.00,2.21,2.41,2.45,3.12,2.62,1.06,3.75,3.38
AAAS,3.05,3.18,2.69,4.21,4.07,3.21,3.76,3.15,3.44,2.55,...,3.46,2.87,3.72,3.36,NaN,2.97,3.22,3.75,3.74,3.02
AACS,2.37,1.42,2.50,3.49,NaN,1.96,NaN,NaN,2.08,NaN,...,NaN,NaN,1.70,NaN,3.44,3.70,2.00,-1.34,NaN,NaN
AAGAB,3.23,3.49,3.32,NaN,1.86,3.44,NaN,3.16,3.06,3.06,...,4.09,3.27,3.32,3.05,4.12,4.48,3.53,2.29,2.75,2.85
AAK1,2.99,1.80,2.59,2.02,2.90,3.52,2.71,3.45,3.85,3.78,...,NaN,NaN,3.70,2.80,3.63,2.98,3.14,2.42,3.58,3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZW10,2.32,1.55,1.84,2.12,2.40,2.64,2.84,2.70,1.13,2.62,...,2.73,2.32,3.48,2.57,3.17,2.51,2.35,1.98,3.11,3.11
ZWILCH,2.38,2.08,1.61,NaN,NaN,2.88,NaN,2.68,2.77,1.70,...,1.84,1.02,1.63,3.00,NaN,2.08,NaN,NaN,NaN,3.48
ZWINT,1.69,1.85,1.74,NaN,NaN,2.11,NaN,2.27,2.20,1.82,...,2.00,NaN,1.97,2.98,NaN,1.96,1.18,NaN,1.63,2.71


In [71]:
#filtering out ALL porteins with nan and transpose - keeping PBRM1 and ARID1A
proteins_to_keep = ["PBRM1", "ARID1A"]
proteins_kept = data1.loc[proteins_to_keep]
data_non_nan = data1.dropna()
data_non_nan = pd.concat([data_non_nan, proteins_kept]).drop_duplicates()
data_non_nan = data_non_nan.T
data_non_nan

protein,ABCB6,ACAT1,ACIN1,ACLY,ACO2,ACOT7,ACTBL2,ACTL6A,ACTN4,ADAR,...,ZC3H11A,ZC3H14,ZC3H4,ZCCHC8,ZFPL1,ZMPSTE24,ZNF207,ZNF638,PBRM1,ARID1A
22RV1,9.44,7.67,6.16,5.73,7.09,4.70,5.33,4.38,6.99,6.50,...,5.36,4.23,3.28,4.46,5.52,4.39,5.57,4.66,3.95,NaN
23132-87,9.63,7.93,6.06,5.31,6.96,6.03,6.33,3.82,7.33,6.50,...,4.16,3.93,3.31,4.57,5.26,5.21,5.85,4.31,-0.97,NaN
42-MG-BA,10.58,6.45,5.70,5.91,6.00,5.45,7.41,4.04,7.70,5.07,...,3.86,4.29,3.12,2.77,5.11,6.27,5.41,3.55,2.34,3.35
451Lu,8.81,8.04,6.37,4.52,8.14,6.24,6.14,4.93,6.67,6.31,...,5.18,4.46,3.71,3.51,5.47,3.78,5.17,4.41,4.06,4.06
5637,5.33,6.05,6.02,2.79,6.21,2.78,5.75,4.48,5.49,6.24,...,4.09,4.44,2.24,3.40,5.86,4.94,4.16,4.49,4.25,3.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,10.25,5.73,5.75,5.78,6.85,6.07,6.44,4.38,6.86,6.44,...,4.74,3.63,4.02,3.74,3.87,4.94,5.98,4.07,3.38,3.66
ZR-75-30,7.89,6.33,5.79,4.81,7.25,4.18,6.96,4.72,7.85,6.65,...,5.14,4.09,3.59,3.52,6.33,5.66,5.26,4.35,3.40,3.32
huH-1,8.30,8.64,6.29,4.55,6.12,2.85,6.45,4.09,6.02,6.54,...,4.44,4.05,3.26,3.49,5.63,5.23,5.53,3.89,3.70,3.86
no-10,9.93,8.05,5.98,6.00,5.72,6.18,6.77,5.22,9.00,5.93,...,4.88,4.04,3.64,3.35,6.52,4.19,5.78,4.18,3.27,3.66


In [72]:
#switching rows and columns
data1_filt = data1_filt.T
data1_filt

protein,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AARS1,AARS2,AARSD1;PTGES3L-AARSD1,...,ZNRD2,ZNRF2,ZPR1,ZRANB2,ZSWIM8,ZW10,ZWILCH,ZWINT,ZYX,ZZZ3
22RV1,2.38,3.05,2.37,3.23,2.99,NaN,2.67,6.43,3.81,3.20,...,3.65,NaN,3.78,4.54,4.85,2.32,2.38,1.69,3.60,0.43
23132-87,NaN,3.18,1.42,3.49,1.80,0.82,3.01,6.05,3.37,3.36,...,3.58,1.92,3.56,3.60,3.74,1.55,2.08,1.85,4.74,NaN
42-MG-BA,2.58,2.69,2.50,3.32,2.59,1.63,2.76,5.85,2.80,3.05,...,4.53,2.28,4.70,4.24,NaN,1.84,1.61,1.74,5.95,NaN
451Lu,NaN,4.21,3.49,NaN,2.02,2.41,3.49,5.53,4.18,NaN,...,2.47,2.18,2.47,4.50,4.24,2.12,NaN,NaN,4.34,2.33
5637,2.79,4.07,NaN,1.86,2.90,NaN,NaN,4.35,3.75,1.58,...,NaN,NaN,NaN,2.50,NaN,2.40,NaN,NaN,1.40,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,3.12,2.97,3.70,4.48,2.98,2.28,3.30,5.91,3.46,2.62,...,5.39,3.94,4.91,4.92,5.09,2.51,2.08,1.96,6.10,NaN
ZR-75-30,2.62,3.22,2.00,3.53,3.14,NaN,0.97,4.22,3.67,0.57,...,4.10,3.36,1.50,3.49,4.30,2.35,NaN,1.18,4.56,NaN
huH-1,1.06,3.75,-1.34,2.29,2.42,NaN,2.22,4.46,3.37,NaN,...,3.78,1.72,2.64,3.31,3.70,1.98,NaN,NaN,3.59,NaN
no-10,3.75,3.74,NaN,2.75,3.58,2.94,2.21,6.73,3.15,2.49,...,4.73,2.96,4.74,5.71,4.70,3.11,NaN,1.63,6.23,2.03


In [73]:
#doing the standard scaler - after transposing - normalises and centers around mean of 0
scaler = StandardScaler()
data1_scaled = pd.DataFrame(scaler.fit_transform(data1_filt), index=data1_filt.index, columns=data1_filt.columns)
data1_scaled

protein,A2M,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AARS1,AARS2,AARSD1;PTGES3L-AARSD1,...,ZNRD2,ZNRF2,ZPR1,ZRANB2,ZSWIM8,ZW10,ZWILCH,ZWINT,ZYX,ZZZ3
22RV1,-0.137746,-0.619731,0.250778,0.165751,0.175627,NaN,-0.137392,0.761121,0.359542,0.148559,...,-0.372580,NaN,0.218864,0.404763,0.916999,-0.399497,0.454948,-0.377226,-0.540186,-0.635745
23132-87,NaN,-0.395180,-0.853789,0.587580,-1.195985,-1.253913,0.296310,0.411460,-0.297479,0.316889,...,-0.447368,-0.938803,0.024958,-0.928088,-0.651135,-1.858296,0.086884,-0.201672,0.162751,NaN
42-MG-BA,0.042071,-1.241563,0.401929,0.311769,-0.285419,-0.478183,-0.022589,0.227427,-1.148619,-0.009249,...,0.567623,-0.546047,1.029744,-0.020615,NaN,-1.308878,-0.489749,-0.322365,0.908851,NaN
451Lu,NaN,1.383953,1.553004,NaN,-0.942409,0.268817,0.908596,-0.067025,0.912036,NaN,...,-1.633305,-0.655146,-0.935758,0.348046,0.055232,-0.778406,NaN,NaN,-0.083893,1.096087
5637,0.230879,1.142129,NaN,-2.056967,0.071892,NaN,NaN,-1.152817,0.269948,-1.555774,...,NaN,NaN,NaN,-2.487808,NaN,-0.247934,NaN,NaN,-1.896731,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,0.527577,-0.757916,1.797171,2.193777,0.164101,0.144317,0.666233,0.282637,-0.163088,-0.461634,...,1.486457,1.264994,1.214836,0.943576,1.256055,-0.039534,0.086884,-0.080978,1.001343,NaN
ZR-75-30,0.078035,-0.326087,-0.179422,0.652477,0.348519,NaN,-2.305904,-1.272439,0.150490,-2.618352,...,0.108206,0.632221,-1.790707,-1.084060,0.139996,-0.342661,NaN,-0.936806,0.051761,NaN
huH-1,-1.324537,0.589389,-4.062845,-1.359326,-0.481363,NaN,-0.711410,-1.051600,-0.297479,NaN,...,-0.233686,-1.157000,-0.785921,-1.339287,-0.707644,-1.043642,NaN,NaN,-0.546352,NaN
no-10,1.094000,0.572115,NaN,-0.613012,0.855670,0.776393,-0.724166,1.037170,-0.625989,-0.598401,...,0.781305,0.195826,1.064999,2.063738,0.705089,1.097192,NaN,-0.443059,1.081502,0.822640


In [74]:
#standard scaler of ther no nan data
scaler = StandardScaler()
no_nan_scaled = pd.DataFrame(scaler.fit_transform(data_non_nan), index=data_non_nan.index, columns=data_non_nan.columns)
no_nan_scaled

protein,ABCB6,ACAT1,ACIN1,ACLY,ACO2,ACOT7,ACTBL2,ACTL6A,ACTN4,ADAR,...,ZC3H11A,ZC3H14,ZC3H4,ZCCHC8,ZFPL1,ZMPSTE24,ZNF207,ZNF638,PBRM1,ARID1A
22RV1,0.339086,0.584458,0.554077,0.363266,1.009232,-0.369939,-1.749049,-0.454913,-0.021696,0.677770,...,1.142044,0.130545,-0.615025,1.326619,0.634928,-0.030758,-0.308983,0.686625,0.751929,NaN
23132-87,0.491141,0.903253,0.279758,-0.059999,0.818395,0.745345,-0.300852,-1.493285,0.279723,0.677770,...,-0.699367,-0.564196,-0.556796,1.526404,0.341235,0.874230,0.118297,-0.021095,-5.720673,NaN
42-MG-BA,1.251418,-0.911428,-0.707788,0.544666,-0.590859,0.258981,1.263201,-1.085353,0.607737,-2.061355,...,-1.159720,0.269493,-0.925578,-1.742820,0.171797,2.044092,-0.553144,-1.557860,-1.366138,-0.291093
451Lu,-0.165098,1.038128,1.130146,-0.856140,2.550605,0.921443,-0.576010,0.564916,-0.305384,0.313830,...,0.865833,0.663180,0.219586,-0.398806,0.578449,-0.703980,-0.919384,0.181111,0.896641,0.811002
5637,-2.950113,-1.401882,0.170031,-2.599589,-0.282585,-1.979974,-1.140807,-0.269490,-1.351483,0.179747,...,-0.806783,0.616864,-2.633619,-0.598591,1.018988,0.576246,-2.460646,0.342875,1.146600,0.593687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YT,0.987322,-1.794245,-0.570629,0.413655,0.656918,0.778888,-0.141551,-0.454913,-0.136944,0.562842,...,0.190648,-1.258937,0.821282,0.018929,-1.228891,0.576246,0.316678,-0.506389,0.002054,0.190103
ZR-75-30,-0.901366,-1.058564,-0.460901,-0.563886,1.244108,-0.805991,0.611512,0.175526,0.740716,0.965091,...,0.804452,-0.193668,-0.013329,-0.380643,1.549894,1.370870,-0.782044,0.059787,0.028366,-0.337661
huH-1,-0.573247,1.773810,0.910691,-0.825907,-0.414703,-1.921275,-0.127069,-0.992641,-0.881625,0.754389,...,-0.269705,-0.286300,-0.653844,-0.435130,0.759183,0.896303,-0.370023,-0.870360,0.423036,0.500553
no-10,0.731228,1.050390,0.060304,0.635365,-1.001892,0.871129,0.336354,1.102644,1.760220,-0.414049,...,0.405480,-0.309458,0.083719,-0.689403,1.764515,-0.251486,0.011477,-0.283963,-0.142658,0.190103


In [75]:
#normal data is the scaled data - need to make it into a tensor first
#now we do the masking
tensor_data = torch.tensor(no_nan_scaled.values, dtype=torch.float32)

tensor_data

tensor([[ 0.3391,  0.5845,  0.5541,  ...,  0.6866,  0.7519,     nan],
        [ 0.4911,  0.9033,  0.2798,  ..., -0.0211, -5.7207,     nan],
        [ 1.2514, -0.9114, -0.7078,  ..., -1.5579, -1.3661, -0.2911],
        ...,
        [-0.5732,  1.7738,  0.9107,  ..., -0.8704,  0.4230,  0.5006],
        [ 0.7312,  1.0504,  0.0603,  ..., -0.2840, -0.1427,  0.1901],
        [ 0.7072,  0.0695, -0.6255,  ..., -0.5671, -0.1295, -1.1914]])

In [76]:
#masking step - after we have converted to a tensor 
nan_mask = torch.isnan(tensor_data)
tensor_no_nan = torch.where(nan_mask, torch.tensor(0.0, dtype=torch.float32), tensor_data)

In [77]:
#creating a data loader for batching
dataset = TensorDataset(tensor_no_nan)
batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [60]:
#defining autoencoder
# Defining Autoencoder model
class AE(nn.Module):
   def __init__(self, input_dim=data1_scaled.shape[1], latent_dim=64):
       super(AE, self).__init__()
       self.encoder = nn.Sequential(
           nn.Linear(input_dim, 4096),
           nn.ReLU(),
           nn.Linear(4096, 1024),
           nn.ReLU(),
           nn.Linear(1024, 256),
           nn.ReLU(),
           nn.Linear(256, latent_dim)
       )
       self.decoder = nn.Sequential(
           nn.Linear(latent_dim, 256),
           nn.ReLU(),
           nn.Linear(256, 1024),
           nn.ReLU(),
           nn.Linear(1024, 4096),
           nn.ReLU(),
           nn.Linear(4096, input_dim)
       )

   def forward(self, x):
       encoded = self.encoder(x)
       decoded = self.decoder(encoded)
       return decoded
    

In [78]:
#defining new autoencoder
# only using data without nans
# need less layers - will try with only two first
class AE_no_nan(nn.Module):
   def __init__(self, input_dim=no_nan_scaled.shape[1], latent_dim=128):
       super(AE_no_nan, self).__init__()
       self.encoder = nn.Sequential(
           nn.Linear(input_dim, 512),
           nn.ReLU(),
           nn.Linear(512, 256),
           nn.ReLU(),
           nn.Linear(256, latent_dim),
           nn.ReLU()
       )
       self.decoder = nn.Sequential(
           nn.Linear(latent_dim, 256),
           nn.ReLU(),
           nn.Linear(256, 512),
           nn.ReLU(),
           nn.Linear(512, input_dim)
       )

   def forward(self, x):
       encoded = self.encoder(x)
       decoded = self.decoder(encoded)
       return decoded

In [79]:
#hyperparameyters
input_dim = no_nan_scaled.shape[1]
latent_dim = 128
learning_rate = 0.001
batch_size = 32
epochs = 100

In [80]:
# Initialize the autoencoder model
model = AE_no_nan(input_dim=input_dim, latent_dim=latent_dim)

# Loss function: Mean Squared Error (MSE)
criterion = nn.MSELoss()

# Optimizer: Adam
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#defining loss funciton (with mask)
def masked_mse_loss(reconstructed, original, mask):
    # Only compute loss on non-NaN (True in mask) values
    loss = F.mse_loss(reconstructed[mask], original[mask], reduction='mean')
    return loss


In [84]:
for epoch in range(epochs):
    model.train()

    # Forward pass
    outputs = model(tensor_no_nan)

    # Compute masked loss (ignore NaNs in both original and reconstructed data)
    loss = masked_mse_loss(outputs, tensor_no_nan, nan_mask)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.0000
Epoch [20/100], Loss: 0.0000
Epoch [30/100], Loss: 0.0000
Epoch [40/100], Loss: 0.0000
Epoch [50/100], Loss: 0.0000
Epoch [60/100], Loss: 0.0000
Epoch [70/100], Loss: 0.0000
Epoch [80/100], Loss: 0.0000
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000


In [86]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for data in train_loader:
        outputs = model(data_no_nan)

        # Compute loss
        loss = masked_mse_loss(outputs, data_no_nan, nan_mask)        
        running_loss += loss.item()

        # Zero the gradients
        optimizer.zero_grad()
        # Backward pass: compute gradients
        loss.backward()
        # Update the weights
        optimizer.step()

    # Print loss for every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [10/100], Loss: 0.0000


KeyboardInterrupt: 

In [109]:
# now i want to simulate knocking out ACTL6 (its in the data set)
# Get the index of the 'ACTL6A' column
actl6a_index = no_nan_scaled.columns.get_loc("PBRM1")
data_knockout = tensor_data.clone()  # if using PyTorch tensors
data_knockout[:, actl6a_index] = 0  # simulate the knockout by setting actl6a to 0

In [110]:
#reconstructing the KO data
model.eval()  # set the model to evaluation mode

with torch.no_grad():
    # Reconstruct the knockout data
    reconstructed_knockout = model(data_knockout)

In [67]:
#reconstruction of og data
with torch.no_grad():
    # Get the reconstruction of the original data
    reconstructed_original = model(data_no_nan)

reconstructed_original

tensor([[-7.6406e-03,  2.5666e-02,  2.4348e-02,  ...,  4.1636e-03,
          5.4911e-06,  1.0426e-04],
        [-1.0619e-02,  2.2438e-02,  2.3603e-02,  ...,  2.1934e-03,
         -1.3265e-04,  1.2324e-04],
        [-1.0432e-02,  1.9412e-02,  2.2943e-02,  ...,  1.1044e-03,
          4.8491e-04,  1.1775e-04],
        ...,
        [-9.1316e-03,  1.8698e-02,  1.9248e-02,  ...,  6.6783e-03,
         -1.0111e-04,  9.5082e-04],
        [-8.9730e-03,  2.5008e-02,  2.1758e-02,  ...,  5.3250e-03,
          2.5148e-04,  8.3852e-04],
        [-9.8803e-03,  2.3489e-02,  2.1984e-02,  ...,  3.1000e-03,
          2.6319e-04,  4.3889e-04]])

In [77]:
#comapring difference in proteins when actl6a is knocked otu
# Compute the difference for each protein across cell lines
delta = reconstructed_original - reconstructed_knockout

# For a summary, compute the mean change across all cell lines for each protein
mean_delta = delta.mean(dim=0)
print("Mean change in protein abundance due to actl6a knockout:")
print(mean_delta)

Mean change in protein abundance due to actl6a knockout:
tensor([ 2.5899e-03,  1.8540e-03, -1.4052e-03, -3.5453e-04, -3.9586e-03,
        -2.1868e-03, -3.6944e-04, -1.6409e-03, -1.3018e-03,  4.1599e-04,
         1.6584e-03, -1.1517e-03,  1.3500e-03, -2.2860e-03,  1.5242e-03,
        -1.8263e-03,  6.1108e-04, -3.6754e-03,  3.7577e-03,  1.5656e-03,
         2.1829e-03, -1.8954e-04, -9.6738e-04, -9.5711e-04, -2.3590e-04,
         4.4385e-04, -1.0559e-04, -1.3636e-03,  5.1790e-04,  8.9632e-04,
         2.3492e-03, -3.3917e-03, -3.8634e-03, -2.9869e-03, -1.7271e-03,
        -5.0623e-04, -1.0416e-03, -2.1603e-03,  8.2041e-04,  1.8962e-03,
         1.9328e-03,  4.0686e-04,  1.8526e-03,  1.8772e-03,  3.0208e-03,
         4.9201e-04,  1.9128e-03,  2.3415e-04,  7.1167e-04,  3.9138e-03,
         4.1213e-03,  3.5002e-03,  3.2197e-03,  2.0575e-04,  3.0015e-03,
        -1.1911e-03, -1.2770e-03,  3.2837e-03, -1.1137e-04, -1.1987e-03,
         2.4720e-03,  3.1112e-03, -5.8852e-03, -2.0486e-05, -1.8996

In [89]:
protein_names = no_nan_scaled.columns
mean_delta_np = mean_delta.numpy() if isinstance(mean_delta, torch.Tensor) else mean_delta
diff_df = pd.DataFrame({'Protein': protein_names, 'Mean Difference': mean_delta_np})
diff_df

,Protein,Mean Difference
0,ABCB6,0.002590
1,ACAT1,0.001854
2,ACIN1,-0.001405
3,ACLY,-0.000355
4,ACO2,-0.003959
...,...,...
817,ZCCHC8,0.002979
818,ZFPL1,-0.000517
819,ZMPSTE24,-0.001953
820,ZNF207,0.002145
